### Импорт необходимых библиотек

In [10]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split
import torch.optim as optim
import torchvision
import pickle
import pandas as pd
import matplotlib as plt
import gc
from tqdm import tqdm
import seaborn as sns
import zipfile
import os

from model import ResNet
from model import ResidualBlock

### Разархировка данных (здесь надо укаазать отдельно для апки с двумя папками людей, а потом отдельно для кремов)

In [ ]:
# Set the path to the archive file
archive_path = '/content/train_resized.zip'

# Set the path to the output folder
output_folder = 'train_resized'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Open the ZIP archive
with zipfile.ZipFile(archive_path, 'r') as zip_ref:
    # Extract all files to the output folder
    zip_ref.extractall(output_folder)

### Делим на train-test-val

In [12]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
dataset = datasets.ImageFolder('/home/julia/Documents/python/DL/train_resized', transform=transform)
train_data, test_data, valid_data = random_split(dataset, (4331, 1247, 610), generator=torch.Generator().manual_seed(42))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Параметры модели

In [5]:
num_classes = 10

num_epochs = 5

batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

# Train the model
total_step = len(train_loader)

### Обучаем! И проверяем на валидации

In [ ]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print (f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(valid_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
            

        print(f'Accuracy of the network on the {len(valid_data)} validation images: {100 * correct / total} %')

### Сохраняем модель

In [ ]:
torch.save(model, '/content/Untitled Folder/model.pt')

### Прогоняем на тесте

In [ ]:
model = torch.load('/content/Untitled Folder/model.pt')
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print(f'Accuracy of the network on the {len(test_loader)} test images: {100 * correct / total} %')
